## 연습문제1

```greeter```의 ```greeting```의 데이터타입을 ```string```에서 ```bytes32```로 변경하세요.

- ```string```에서 ```bytes32```로 변경하면 gas의 차이가 있는지 확인

- 노드에서 배포

- 노드에서 인사를 설정하고 ```setGreeting("Hello World!")```, ```greet()```을 출력하세요.

In [3]:
%%writefile ../src/e8-contract-greeter.sol
pragma solidity 0.6.4;

contract Greeter {
    bytes32 greeting;
    
    constructor() public {
        greeting = "Hello";
    }
    
    function setGreeting (bytes32 _greeting) public {
        greeting = _greeting;
    }
    
    function greet() view public returns (bytes32) {
        return greeting;
    }
}

Overwriting ../src/e8-contract-greeter.sol


In [4]:
!solc --abi --bin ../src/e8-contract-greeter.sol


======= ../src/e8-contract-greeter.sol:Greeter =======
Binary:
608060405234801561001057600080fd5b507f48656c6c6f00000000000000000000000000000000000000000000000000000060008190555060c7806100466000396000f3fe6080604052348015600f57600080fd5b506004361060325760003560e01c806350513b4f146037578063cfae3217146062575b600080fd5b606060048036036020811015604b57600080fd5b8101908080359060200190929190505050607e565b005b60686088565b6040518082815260200191505060405180910390f35b8060008190555050565b6000805490509056fea26469706673582212209783fe7472d18828b662f67d756e46d27e43599a41f4654582caa72d85acf53764736f6c63430006040033
Contract JSON ABI
[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"greet","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes32","name":"_greeting","type":"bytes32"}],"name":"setGreeting","outputs":[],"stateMutability":"nonpayable","type":"function"}]


In [5]:
!solc --gas ../src/e8-contract-greeter.sol


======= ../src/e8-contract-greeter.sol:Greeter =======
Gas estimation:
construction:
   20107 + 39800 = 59907
external:
   greet():	1013
   setGreeting(bytes32):	20220


- string의 경우    
배열의 크기를 알 수 없기 때문에 gas 비용이 infinite로 산정된다.

- bytes32의 경우     
글자수가 제한되므로 gas 비용이 한정되어 산정된다.

### 배포

In [6]:
%%writefile ../src/e8-contract-greeterDeploy.js
var Web3 = require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"greet","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes32","name":"_greeting","type":"bytes32"}],"name":"setGreeting","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _bin = "608060405234801561001057600080fd5b507f48656c6c6f00000000000000000000000000000000000000000000000000000060008190555060c7806100466000396000f3fe6080604052348015600f57600080fd5b506004361060325760003560e01c806350513b4f146037578063cfae3217146062575b600080fd5b606060048036036020811015604b57600080fd5b8101908080359060200190929190505050607e565b005b60686088565b6040518082815260200191505060405180910390f35b8060008190555050565b6000805490509056fea26469706673582212209783fe7472d18828b662f67d756e46d27e43599a41f4654582caa72d85acf53764736f6c63430006040033";
var _contract = new web3.eth.Contract(_abi);

_contract
    .deploy({ data: "0x"+_bin })
    .send({
     from: '0xCE966b32cAd5E30E5A20F425bac553F07641eEe3',
     gas: '1000000'
    })
    .then(function(newContractInstance){
        console.log("contract address: "+newContractInstance.options.address)
    });

Writing ../src/e8-contract-greeterDeploy.js


In [7]:
!node ../src/e8-contract-greeterDeploy.js

contract address: 0x81E522f375be14670DED3aadCe7014ff3897e347


### 사용

In [33]:
%%writefile ../src/e8-contract-greeterUse.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var _abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"greet","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes32","name":"_greeting","type":"bytes32"}],"name":"setGreeting","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var greeter = new web3.eth.Contract(_abi, "0x81E522f375be14670DED3aadCe7014ff3897e347");

// 파라미터로 전달할 문자열을 16진수로 변환 (utf8 -> hex)
var str = web3.utils.toHex("Hello World!");
greeter.methods.setGreeting(str).send({from: "0xCE966b32cAd5E30E5A20F425bac553F07641eEe3", gas: 1000000});
greeter.methods.greet().call().then(function(value) { console.log(web3.utils.hexToUtf8(value)); });     // 결과 출력시 hex -> utf8

Overwriting ../src/e8-contract-greeterUse.js


In [34]:
!node ../src/e8-contract-greeterUse.js

Hello World!


## 연습문제2

어떤 수에 7을 곱하는 스마트 컨트랙을 만든다.

컨트랙은 ```Multiply7```, 함수는:

function multiply(uint input) public pure returns (uint)

컨트랙은 Solidity로, 클라이언트는 노드로 프로그램한다.

- 8을 인자로 넘기면, 7을 곱해서 56을 출력하세요.

### 1. 개발

In [35]:
%%writefile ../src/e8-contract-multiply7.sol
pragma solidity 0.6.4;

contract Multiply7 {
    function multiply(uint input) public pure returns(uint) {
        return 7 * input;
    }
}

Writing ../src/e8-contract-multiply7.sol


### 2. 컴파일

In [36]:
!solc --abi --bin --gas ../src/e8-contract-multiply7.sol


======= ../src/e8-contract-multiply7.sol:Multiply7 =======
Gas estimation:
construction:
   87 + 35000 = 35087
external:
   multiply(uint256):	286
Binary:
6080604052348015600f57600080fd5b5060af8061001e6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c8063c6888fa114602d575b600080fd5b605660048036036020811015604157600080fd5b8101908080359060200190929190505050606c565b6040518082815260200191505060405180910390f35b600081600702905091905056fea264697066735822122064ab053a01a7c9ec0312bffdf91dc6edfe2e6951ca03567ccfaa965decd619fa64736f6c63430006040033
Contract JSON ABI
[{"inputs":[{"internalType":"uint256","name":"input","type":"uint256"}],"name":"multiply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"pure","type":"function"}]


### 3. 배포

In [37]:
%%writefile ../src/e8-contract-multiply7Deploy.js
var Web3 = require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

var _abi = [{"inputs":[{"internalType":"uint256","name":"input","type":"uint256"}],"name":"multiply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"pure","type":"function"}];
var _bin = "6080604052348015600f57600080fd5b5060af8061001e6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c8063c6888fa114602d575b600080fd5b605660048036036020811015604157600080fd5b8101908080359060200190929190505050606c565b6040518082815260200191505060405180910390f35b600081600702905091905056fea264697066735822122064ab053a01a7c9ec0312bffdf91dc6edfe2e6951ca03567ccfaa965decd619fa64736f6c63430006040033";
var _contract = new web3.eth.Contract(_abi);

_contract
    .deploy({ data: "0x"+_bin })
    .send({
     from: '0xCE966b32cAd5E30E5A20F425bac553F07641eEe3',
     gas: '1000000'
    })
    .then(function(newContractInstance){
        console.log("contract address: "+newContractInstance.options.address)
    });

Writing ../src/e8-contract-multiply7Deploy.js


In [38]:
!node ../src/e8-contract-multiply7Deploy.js

contract address: 0xB6D705C1693B197A3DdD133e1B8e47b05cB68aE0


### 4. 사용

In [41]:
%%writefile ../src/e8-contract-multiply7Use.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var _abi = [{"inputs":[{"internalType":"uint256","name":"input","type":"uint256"}],"name":"multiply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"pure","type":"function"}];
var multiply7 = new web3.eth.Contract(_abi, "0xB6D705C1693B197A3DdD133e1B8e47b05cB68aE0");
multiply7.methods.multiply(8).call().then(function(value) { console.log(value); });

Overwriting ../src/e8-contract-multiply7Use.js


In [42]:
!node ../src/e8-contract-multiply7Use.js

56
